In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("xsum")
dataset["train"][100]


Using custom data configuration default
Reusing dataset xsum (C:\Users\User\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

{'document': 'South Wales Police are investigating the robbery at Lloyds Bank on Cowbridge Road East in the Canton area of the city at 12:30 BST on Tuesday.\nOfficers are trying to trace a suspect captured on CCTV cameras within the bank.\nThe man is described as 50-60 years of age, 5ft 9ins (1.75m) to 6ft (1.82m) tall with black hair that is greying. He was wearing black clothing.\nDetective Sergeant Andy Miles from Fairwater CID said: "The CCTV is clear and I am confident that members of the public will know his identity.\n"I can confirm there have been no reports of any injuries as a result of the incident."',
 'summary': 'A large amount of cash has been stolen in a daylight bank robbery in Cardiff.',
 'id': '32138822'}

In [3]:
from transformers import BartTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn", output_attentions=True)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")



In [4]:
def tokenize_function(examples):
    return tokenizer(examples["document"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934\cache-fca1f4eb3d05ee18.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934\cache-88c1b8b9890a9b7d.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934\cache-2b11f5ebb4d93bc5.arrow


In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at C:\Users\User\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934\cache-aa92736fc442d73d.arrow
Loading cached shuffled indices for dataset at C:\Users\User\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934\cache-fcd95db7070a4808.arrow


In [19]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print("Using device: ", device)
model = model.to(device)


Using device:  cpu


In [20]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir = True,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    load_best_model_at_end = True,
    evaluation_strategy='steps',
    save_total_limit=3,)

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [21]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")


In [22]:
model.base_model.parameters()

<generator object Module.parameters at 0x000001A0D817AF90>

In [23]:
encoder_to_freeze = np.arange(2,130)
decoder_to_freeze = np.arange(196, 405)

to_freeze = np.concatenate((encoder_to_freeze,decoder_to_freeze))
to_freeze #all encoder and decoder layers to freeze indexes except the last four

array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 196, 197,
       198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210,
       211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223,
       224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236,
       237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 24

In [24]:
for idx, attr  in enumerate(model.named_parameters()):
    if idx in to_freeze:
        attr[1].requires_grad = False
    

In [25]:
for idx, attr  in enumerate(model.named_parameters()):
    print(attr[1].requires_grad)
    break

True


In [26]:
def compute_metrics(eval_pred):
    logits , labels = eval_pred
    predictions = np.argmax(logits, axis = -1)
    return metric.compute(predictions=predictions, references = labels)

In [27]:
from transformers import Trainer

training_args = TrainingArguments(output_dir="test_trainer", 
#                                   evaluation_strategy='epoch',
                                  eval_accumulation_steps=1,
                                  do_train=True,
                                  do_eval=True,
                                  evaluation_strategy="steps",
                                  eval_steps=1000,
                                  logging_dir="logs",
                                  num_train_epochs=1,
                                  gradient_accumulation_steps=2,
                                  per_device_train_batch_size = 2,
                                  per_device_eval_batch_size =2)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    
#     compute_metrics=compute_metrics,
)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 4.47 GiB already allocated; 0 bytes free; 4.48 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.train()

# Pytorch Training

In [9]:
del model 
del trainer

torch.cuda.empty_cache()

NameError: name 'trainer' is not defined

In [12]:
tokenized_datasets = tokenized_datasets.remove_columns(['document', 'summary'])

In [13]:
tokenized_datasets = tokenized_datasets.remove_columns(['id'])

In [14]:
tokenized_datasets['train']

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 204045
})

In [15]:
tokenized_datasets.set_format("torch")

In [16]:
tokenized_datasets['train']

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 204045
})

In [17]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

Loading cached shuffled indices for dataset at C:\Users\User\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934\cache-538f7aef17d9d9ce.arrow
Loading cached shuffled indices for dataset at C:\Users\User\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934\cache-c3be7224a6d6e1f4.arrow


In [18]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size = 1)
eval_dataloader = DataLoader(small_eval_dataset, batch_size = 1)


In [19]:
from torch.optim import AdamW
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn", output_attentions=True)
learning_rate = 1e-1
optimizer = AdamW(model.parameters(), lr=learning_rate)

In [22]:
 model = model.to(device)

In [26]:
from tqdm.auto import tqdm
from transformers import get_scheduler
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
def train_model(num_epochs, dataloader):
    val_acc_history = []
    loss_acc_history = []
    progress_bar = tqdm(range(num_training_steps))
    
    model.train()
    for epoch in range(num_epochs):
        for batch in dataloader: 
            batch = {k: v.to(device) for k, v in batch.items()} 
            outputs = model(**batch) 
            if outputs.loss != None:
                loss = outputs.loss
                loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            

In [27]:

train_model(2, train_dataloader)

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [21]:
device = 'cpu'